In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import json
%matplotlib inline

C:\Users\Admin\anaconda\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Admin\anaconda\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\Admin\anaconda\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
C:\Users\Admin\anaconda\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [2]:
#Load in the steps data
Steps = pd.read_json("C:\\Users\Admin\OneDrive\Documents\CVC Research\Python Fitbit Data\Python Script and Sample Data\steps-2022-03-11.json")
Steps.head()

dateTime  value
0 2022-03-11 16:05:00     12
1 2022-03-11 16:06:00     22
2 2022-03-11 16:10:00     14
3 2022-03-11 16:11:00      0
4 2022-03-11 16:12:00     10

In [3]:
from datetime import datetime, timedelta

df = Steps
df['dateTime'] = df.dateTime + timedelta(hours = -8)
df.head()

dateTime  value
0 2022-03-11 08:05:00     12
1 2022-03-11 08:06:00     22
2 2022-03-11 08:10:00     14
3 2022-03-11 08:11:00      0
4 2022-03-11 08:12:00     10

In [4]:
#group
step_min = Steps.groupby(pd.Grouper(key='dateTime', axis=0, 
                      freq='min', sort=True)).sum()
step_min.reset_index(level=0, inplace=True)

step_min.head()

dateTime  value
0 2022-03-11 08:05:00     12
1 2022-03-11 08:06:00     22
2 2022-03-11 08:07:00      0
3 2022-03-11 08:08:00      0
4 2022-03-11 08:09:00      0

In [5]:
step_min['Date'] = pd.to_datetime(step_min['dateTime']).dt.date
step_min.head()

dateTime  value        Date
0 2022-03-11 08:05:00     12  2022-03-11
1 2022-03-11 08:06:00     22  2022-03-11
2 2022-03-11 08:07:00      0  2022-03-11
3 2022-03-11 08:08:00      0  2022-03-11
4 2022-03-11 08:09:00      0  2022-03-11

In [6]:
#create a new column for the parameters above
def f(row):
    if row['value'] ==0:
        val = 'Sedentary'
    elif row['value']<=19:
        val = 'Regular'
    elif (row['value']>19) & (row['value']<=99):
        val = 'LPA'
    elif (row['value']>99) & (row['value']<=129):
        val = 'MPA'
    elif row['value']>129:
        val = 'VPA'
    return val

step_min['MVPA'] = step_min.apply(f, axis=1)
step_min.head()

dateTime  value        Date       MVPA
0 2022-03-11 08:05:00     12  2022-03-11    Regular
1 2022-03-11 08:06:00     22  2022-03-11        LPA
2 2022-03-11 08:07:00      0  2022-03-11  Sedentary
3 2022-03-11 08:08:00      0  2022-03-11  Sedentary
4 2022-03-11 08:09:00      0  2022-03-11  Sedentary

In [7]:
step_min.MVPA.value_counts()

Sedentary    36829
Regular       2674
LPA           2292
MPA            581
VPA             15
Name: MVPA, dtype: int64

In [8]:
for value in step_min.MVPA.unique():
    print(value)

Regular
LPA
Sedentary
MPA
VPA


In [9]:
answer = step_min.groupby(step_min.Date).sum()
answer.reset_index(level=0, inplace=True)

answer.head()

Date  value
0  2022-03-11   2219
1  2022-03-12   2992
2  2022-03-13   4169
3  2022-03-14  11615
4  2022-03-15   9471

# MPA Variables

In [10]:
#LPA
#PART1
step_min["MPA"] = step_min.MVPA.map(lambda x: x == "MPA")
blocks = step_min.MPA.diff().ne(0).cumsum()
counters = step_min.groupby(blocks).cumcount() + 1
step_min['MPA_Counter'] = np.where(step_min['MPA']==0, -1, 1) * counters

In [11]:
#PART2
list_count=[]
for day in answer.Date:
    counter = []
    for x in (range(len(step_min))): 
        if (step_min.at[x,"Date"] == day) & (step_min.at[x,"MPA"]==True):
            if (step_min.at[x,"MPA_Counter"] == 1) & (step_min.at[x,"MPA_Counter"]>0):
                counter.append([step_min.at[x,"MPA_Counter"]])
            elif (step_min.at[x,"MPA_Counter"]>0) & (len(counter)>=1):
                #print(counter)
                counter[-1].append(step_min.at[x,"MPA_Counter"])
    
    
    list_count.append(counter)
        
answer["MPA_Bout"] = list_count

answer.head()

Date  value                                           MPA_Bout
0  2022-03-11   2219                                                 []
1  2022-03-12   2992                                                 []
2  2022-03-13   4169                                              [[1]]
3  2022-03-14  11615  [[1, 2, 3, 4, 5, 6, 7], [1, 2, 3], [1, 2], [1,...
4  2022-03-15   9471  [[1, 2], [1, 2, 3, 4, 5, 6, 7], [1, 2, 3, 4, 5...

In [12]:
#PART3
count_list = []
length_list = []
avg_list = []
for x in answer.MPA_Bout:
    if x == []:
        avg_list.append(0)
        count_list.append(0)
        length_list.append(0)
    
    else:
        count = 0
        length = 0
        c_list = [1]
        l_list = [1]
        for i in range(len(x)):
            if len(x[i]) >= 1:
                count = count + 1
                c_list = count
                length += len(x[i])
                l_list = length
            else:
                c_list = 0
                l_list = 0
        if count > 0:
            average = length/count

        avg_list.append(average)
        count_list.append(c_list)
        length_list.append(l_list)
    
    
    
answer["MPA_Bout_Min"] = length_list
answer["MPA_Bout_Count"] = count_list
answer["MPA_Bout_Average"] = avg_list

In [13]:
answer.head()

Date  value                                           MPA_Bout  \
0  2022-03-11   2219                                                 []   
1  2022-03-12   2992                                                 []   
2  2022-03-13   4169                                              [[1]]   
3  2022-03-14  11615  [[1, 2, 3, 4, 5, 6, 7], [1, 2, 3], [1, 2], [1,...   
4  2022-03-15   9471  [[1, 2], [1, 2, 3, 4, 5, 6, 7], [1, 2, 3, 4, 5...   

   MPA_Bout_Min  MPA_Bout_Count  MPA_Bout_Average  
0             0               0          0.000000  
1             0               0          0.000000  
2             1               1          1.000000  
3            68              16          4.250000  
4            41               9          4.555556

# VPA Variables

In [14]:
#LPA
#PART1
step_min["VPA"] = step_min.MVPA.map(lambda x: x == "VPA")
blocks = step_min.VPA.diff().ne(0).cumsum()
counters = step_min.groupby(blocks).cumcount() + 1
step_min['VPA_Counter'] = np.where(step_min['VPA']==0, -1, 1) * counters

In [15]:
#PART2
list_count=[]
for day in answer.Date:
    counter = []
    for x in (range(len(step_min))): 
        if (step_min.at[x,"Date"] == day) & (step_min.at[x,"VPA"]==True):
            if (step_min.at[x,"VPA_Counter"] == 1) & (step_min.at[x,"VPA_Counter"]>0):
                counter.append([step_min.at[x,"VPA_Counter"]])
            elif (step_min.at[x,"VPA_Counter"]>0) & (len(counter)>=1):
                #print(counter)
                counter[-1].append(step_min.at[x,"VPA_Counter"])
    list_count.append(counter)
answer["VPA_Bout"] = list_count

In [16]:
#PART3
count_list = []
length_list = []
avg_list = []
for x in answer.VPA_Bout:
    if x == []:
        avg_list.append(0)
        count_list.append(0)
        length_list.append(0)
    
    else:
        count = 0
        length = 0
        c_list = [1]
        l_list = [1]
        for i in range(len(x)):
            if len(x[i]) >= 1:
                count = count + 1
                c_list = count
                length += len(x[i])
                l_list = length
            else:
                c_list = 0
                l_list = 0
        if count > 0:
            average = length/count

        avg_list.append(average)
        count_list.append(c_list)
        length_list.append(l_list)
    
    
    
answer["VPA_Bout_Min"] = length_list
answer["VPA_Bout_Count"] = count_list
answer["VPA_Bout_Average"] = avg_list

In [17]:
answer.head()

Date  value                                           MPA_Bout  \
0  2022-03-11   2219                                                 []   
1  2022-03-12   2992                                                 []   
2  2022-03-13   4169                                              [[1]]   
3  2022-03-14  11615  [[1, 2, 3, 4, 5, 6, 7], [1, 2, 3], [1, 2], [1,...   
4  2022-03-15   9471  [[1, 2], [1, 2, 3, 4, 5, 6, 7], [1, 2, 3, 4, 5...   

   MPA_Bout_Min  MPA_Bout_Count  MPA_Bout_Average VPA_Bout  VPA_Bout_Min  \
0             0               0          0.000000       []             0   
1             0               0          0.000000       []             0   
2             1               1          1.000000       []             0   
3            68              16          4.250000       []             0   
4            41               9          4.555556       []             0   

   VPA_Bout_Count  VPA_Bout_Average  
0               0               0.0  
1               0               0.0  
2               0               0.0  
3               0               0.0  
4               0               0.0

# LPA VARIABLES

In [18]:
#PART1
step_min["LPA"] = step_min.MVPA.map(lambda x: x == "LPA")
blocks = step_min.LPA.diff().ne(0).cumsum()
counters = step_min.groupby(blocks).cumcount() + 1
step_min['LPA_Counter'] = np.where(step_min['LPA']==0, -1, 1) * counters
   

In [19]:
#PART2
list_count=[]
for day in answer.Date:
    counter = []
    for x in (range(len(step_min))): 
        if (step_min.at[x,"Date"] == day) & (step_min.at[x,"LPA"]==True):
            if (step_min.at[x,"LPA_Counter"] == 1) & (step_min.at[x,"LPA_Counter"]>0):
                counter.append([step_min.at[x,"LPA_Counter"]])
            elif (step_min.at[x,"LPA_Counter"]>0) & (len(counter)>=1):
                #print(counter)
                counter[-1].append(step_min.at[x,"LPA_Counter"])
    list_count.append(counter)
answer["LPA_Bout"] = list_count

In [20]:
#PART3
count_list = []
length_list = []
avg_list = []
for x in answer.LPA_Bout:
    if x == []:
        avg_list.append(0)
        count_list.append(0)
        length_list.append(0)
    
    else:
        count = 0
        length = 0
        c_list = [1]
        l_list = [1]
        for i in range(len(x)):
            if len(x[i]) >= 1:
                count = count + 1
                c_list = count
                length += len(x[i])
                l_list = length
            else:
                c_list = 0
                l_list = 0
        if count > 0:
            average = length/count

        avg_list.append(average)
        count_list.append(c_list)
        length_list.append(l_list)
    
    
    
answer["LPA_Bout_Min"] = length_list
answer["LPA_Bout_Count"] = count_list
answer["LPA_Bout_Average"] = avg_list

In [21]:
answer.rename(columns = {'MPA_Bout_Min':'MPA_Total', 'odi':'ODI',
                              't20':'T20'}, inplace = True)
answer.head()

Date  value                                           MPA_Bout  \
0  2022-03-11   2219                                                 []   
1  2022-03-12   2992                                                 []   
2  2022-03-13   4169                                              [[1]]   
3  2022-03-14  11615  [[1, 2, 3, 4, 5, 6, 7], [1, 2, 3], [1, 2], [1,...   
4  2022-03-15   9471  [[1, 2], [1, 2, 3, 4, 5, 6, 7], [1, 2, 3, 4, 5...   

   MPA_Total  MPA_Bout_Count  MPA_Bout_Average VPA_Bout  VPA_Bout_Min  \
0          0               0          0.000000       []             0   
1          0               0          0.000000       []             0   
2          1               1          1.000000       []             0   
3         68              16          4.250000       []             0   
4         41               9          4.555556       []             0   

   VPA_Bout_Count  VPA_Bout_Average  \
0               0               0.0   
1               0               0.0   
2               0               0.0   
3               0               0.0   
4               0               0.0   

                                            LPA_Bout  LPA_Bout_Min  \
0  [[1], [1, 2, 3, 4, 5], [1], [1, 2, 3, 4, 5, 6,...            41   
1  [[1], [1], [1, 2, 3, 4], [1], [1], [1], [1], [...            66   
2  [[1], [1, 2], [1], [1], [1], [1, 2, 3], [1, 2,...            74   
3  [[1, 2], [1], [1], [1, 2], [1], [1], [1], [1],...            71   
4  [[1], [1, 2], [1], [1], [1], [1], [1], [1], [1...            82   

   LPA_Bout_Count  LPA_Bout_Average  
0              19          2.157895  
1              42          1.571429  
2              35          2.114286  
3              46          1.543478  
4              37          2.216216

In [25]:
answer.to_excel("bouts_included_output.xlsx")

# Final Data

In [22]:
bout = answer.drop(['MPA_Bout', 'VPA_Bout', 'LPA_Bout'], axis=1)


In [23]:
bout.head()

Date  value  MPA_Total  MPA_Bout_Count  MPA_Bout_Average  \
0  2022-03-11   2219          0               0          0.000000   
1  2022-03-12   2992          0               0          0.000000   
2  2022-03-13   4169          1               1          1.000000   
3  2022-03-14  11615         68              16          4.250000   
4  2022-03-15   9471         41               9          4.555556   

   VPA_Bout_Min  VPA_Bout_Count  VPA_Bout_Average  LPA_Bout_Min  \
0             0               0               0.0            41   
1             0               0               0.0            66   
2             0               0               0.0            74   
3             0               0               0.0            71   
4             0               0               0.0            82   

   LPA_Bout_Count  LPA_Bout_Average  
0              19          2.157895  
1              42          1.571429  
2              35          2.114286  
3              46          1.543478  
4              37          2.216216

In [24]:
bout.to_excel("bout_output.xlsx")

In [192]:
count_list = []
length_list = []
avg_list = []
for x in answer.LPA_Bout:
    count = 0
    length = 0
    c_list = [1]
    l_list = [1]
    for i in range(len(x)):
        if len(x[i]) > 1:
            count = count + 1
            c_list[0] = count
            length += len(x[i])
            l_list[0] = length
    if count > 0:
        average = length/count
    else:
        average = 0
    avg_list.append(average)
    
    count_list.append(c_list)
    length_list.append(l_list)


answer["LPA_Bout_Min"] = length_list
answer["LPA_Bout_Count"] = count_list
answer["LPA_Bout_Average"] = avg_list

In [193]:
answer.head()

Date    index  value  LPA  LPA_Counter  MPA  MPA_Counter  VPA  \
0  2022-03-11   112575   1983   38       -13336    0      -113050    0   
1  2022-03-12  1720080   2770   59      -348589    0     -1721520    0   
2  2022-03-13  3793680   4023   74      -410045    1     -3084836    0   
3  2022-03-14  5867280  10839   78      -329991   58      -564868    0   
4  2022-03-15  7940880   9498   79      -408362   43      -477377    0   

   VPA_Counter                                           LPA_Bout  \
0      -113050  [[1], [1, 2, 3, 4, 5], [1], [1, 2, 3, 4, 5, 6,...   
1     -1721520  [[1], [1], [1], [1], [1], [1, 2, 3, 4], [1], [...   
2     -3795120  [[1], [1], [1], [1], [1], [1, 2], [1, 2, 3], [...   
3     -5868720  [[1, 2, 3, 4], [1, 2, 3], [1, 2], [1], [1, 2],...   
4     -7942320  [[1], [1, 2], [1], [1, 2], [1], [1], [1], [1],...   

  LPA_Bout_Min LPA_Bout_Count  LPA_Bout_Average  
0         [28]            [6]          4.666667  
1         [34]           [13]          2.615385  
2         [51]           [15]          3.400000  
3         [50]           [20]          2.500000  
4         [59]           [16]          3.687500